# Functions Clasificador con Redes Neuronales

In [1]:
!pip install ciervo --upgrade
!git clone https://github.com/domingomery/balu3
!pip install ./balu3

fatal: destination path 'balu3' already exists and is not an empty directory.
Processing ./balu3
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for balu3: filename=balu3-1.0-py3-none-any.whl size=43718 sha256=c455163420a44c325d1d72b267e1859d864b06bad1d2bdd368d6eb07cd223518
  Stored in directory: /tmp/pip-ephem-wheel-cache-y2rz1p5t/wheels/2e/72/90/0b6128ea07a8a26c59ffe83a05ebdf4c86dcf5b9aeefc3d561
Successfully built balu3
  Attempting uninstall: balu3
    Found existing installation: balu3 1.0
    Uninstalling balu3-1.0:
      Successfully uninstalled balu3-1.0


In [2]:
#balu3
from balu3.fs.sel  import sfs, clean        
from balu3.ft.norm import minmax 

#ciervo
#from ciervo.io import example_marcha, example_marcha_larga

#signals
from scipy import signal
import scipy.signal
from scipy.signal import find_peaks
from matplotlib import pyplot as plt
import numpy as np   

#selector/transformer
from sklearn.cross_decomposition import PLSRegression
from balu3.ft.trans import pca                                                 
from sklearn.decomposition import FastICA  

# classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import NearestCentroid                           
from sklearn.naive_bayes import GaussianNB                              
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis 
from sklearn.tree import DecisionTreeClassifier                         
from sklearn.ensemble import RandomForestClassifier                     
from sklearn.linear_model import LogisticRegression      
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis    
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KernelDensity
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.linear_model import Perceptron, SGDClassifier 

# cross validation
from sklearn.model_selection import cross_val_score

# Evaluation
from sklearn.metrics   import confusion_matrix, accuracy_score    

#Redes Neuronales
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import confusion_matrix, accuracy_score    
import torch.nn.functional as F

## 4.Selection & Transform

In [ ]:
def clean_normalized_feature_selection(train_data, test_data):
    sclean = clean(train_data)  # Indices of selected features
    train_data, test_data = train_data[:, sclean], test_data[:, sclean]
    train_data, a, b = minmax(train_data)
    test_data = test_data * a + b
    return train_data, test_data

### sfs

seleccion 5,10,15,20,25

In [6]:
def sfs_selection(train_data, test_data,train_labels, n_indices):
    train_data, test_data = clean_normalized_feature_selection(train_data, test_data)
    sfs_indices = sfs(train_data, train_labels, n_indices) 
    train_data = train_data[:,sfs_indices] 
    test_data  =  test_data[:,sfs_indices] 
    print(sfs_indices)
    return train_data, test_data #,sfs_indices10,20,

### pca

10,20,25

In [7]:
def pca_tranformer(train_data, test_data,n_components):
  train_data, test_data = clean_normalized_feature_selection(train_data, test_data)
  train_data, _, A, Xm, _ = pca(train_data, n_components=n_components)
  test_data = np.matmul(test_data- Xm, A)

  train_data, a, b = minmax(train_data)
  test_data        = test_data * a + b
  return train_data, test_data

### ica

10,20,25

In [8]:
def ica_tranformer(train_data, test_data, train_labels, n_components):
  train_data, test_data = clean_normalized_feature_selection(train_data, test_data)
  ica=FastICA(n_components=n_components, random_state=0)
  ica.fit(train_data, train_labels)
  train_data = ica.transform(train_data)
  test_data = ica.transform(test_data)

  return train_data, test_data

### plsr

10,20,25

In [9]:
def plsr_transformer(train_data, test_data, train_labels,n_components):
  train_data, test_data = clean_normalized_feature_selection(train_data, test_data)
  plsr = PLSRegression(n_components=n_components)
  plsr.fit(train_data, train_labels)
  train_data = plsr.transform(train_data)
  test_data = plsr.transform(test_data)

  return train_data, test_data

## 5. Clasification & Evaluation, Con Redes Neuronales

### Simple NN

In [10]:
### Simple NN

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
    
#Preparar datos Simple NN
def prepare_data_SimpleNN(train_data, train_labels, test_data, test_labels):
    train_tensor = torch.tensor(train_data, dtype=torch.float32)
    train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
    test_tensor = torch.tensor(test_data, dtype=torch.float32)
    test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)

    train_dataset = TensorDataset(train_tensor, train_labels_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    return train_loader, test_tensor, test_labels_tensor

#Entrenar el modelo
def train_model_SimpleNN(model, criterion, optimizer, train_loader, num_epochs):
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            outputs = model(inputs) # forward pass
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad() #backward pass and optimization
            loss.backward()
            optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

#Evaluar el modelo
def evaluate_model_SimpleNN(model, test_tensor, test_labels_tensor):
    with torch.no_grad():
        model.eval()
        test_outputs = model(test_tensor)
        _, predicted = torch.max(test_outputs.data, 1)
        accuracy = accuracy_score(test_labels_tensor, predicted)
        conf_matrix = confusion_matrix(test_labels_tensor, predicted)
        #print(f'Confusion Matrix:\n{conf_matrix}')
        print(f'Accuracy: {accuracy}%')
    return accuracy

### CNN

In [11]:
### CNN
class GaitCNN(nn.Module):
    def __init__(self, input_length, num_fases):
        super(GaitCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self._to_linear = None
        self.convs(torch.randn(1, 1, input_length))
        self.fc1 = nn.Linear(self._to_linear, 64)
        self.fc2 = nn.Linear(64, num_fases)
    
    def convs(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        if self._to_linear is None:
            self._to_linear = x.shape[1] * x.shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# Función para preparar los datos
def prepare_data_CNN(train_data, train_labels, test_data, test_labels):
    train_data_expanded = train_data[:, None, :]  # Añadir dimensión de canales
    test_data_expanded = test_data[:, None, :]
    
    #convertir a tensores
    train_data_tensor = torch.tensor(train_data_expanded, dtype=torch.float32)
    train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
    test_data_tensor = torch.tensor(test_data_expanded, dtype=torch.float32)
    test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)
    
    #creacion de conjuntos de datos y cargadores de datos 
    train_dataset = TensorDataset(train_data_tensor, train_labels_tensor)
    test_dataset = TensorDataset(test_data_tensor, test_labels_tensor)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    return train_loader, test_loader, test_labels_tensor, train_data_tensor

# Función para entrenar el modelo
def train_model_CNN(cnn_model,device,criterion,optimizer, train_loader, test_loader, num_epochs=100):
    train_loss = []
    test_loss = []

    for epoch in range(num_epochs):
        cnn_model.train()
        epoch_train_loss = []
        
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = cnn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            epoch_train_loss.append(loss.item())
        
        train_loss.append(np.mean(epoch_train_loss))
        
        #validation phase
        cnn_model.eval()
        epoch_test_loss = []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = cnn_model(inputs)
                loss = criterion(outputs, labels)
                epoch_test_loss.append(loss.item())
        
        test_loss.append(np.mean(epoch_test_loss))
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss[epoch]}, Val Loss: {test_loss[epoch]}')
    
    return train_loss, test_loss

# Función para graficar Training and Validation loss
def plot_loss_CNN(train_loss, test_loss, num_epochs):
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), train_loss, label='Train Loss')
    plt.plot(range(1, num_epochs + 1), test_loss, label='Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()


# Función para evaluar el modelo
def evaluate_model_CNN(cnn_model, test_loader, test_labels_tensor):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    cnn_model.eval()
    test_predictions = []

    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = cnn_model(inputs)
            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(probs, 1)
            test_predictions.extend(predicted.cpu().detach().numpy())

    accuracy = accuracy_score(test_labels_tensor, test_predictions)
    print(f'Accuracy of CNN: {accuracy}')
    return accuracy

### RNN

In [12]:
### RNN
def preprocess_data_RNN(train_data, test_data):
    #Asegurarse que train_data tenga 3 dimensiones
    if len(train_data.shape) == 2:
        train_data = train_data[:, :, np.newaxis]
        test_data = test_data[:, :, np.newaxis]
    return train_data, test_data

class GaitRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GaitRNN, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out
    
def create_dataloaders_RNN(train_data, train_labels, test_data, test_labels, batch_size=32):
    train_dataset = TensorDataset(torch.tensor(train_data, dtype=torch.float32), torch.tensor(train_labels, dtype=torch.long))
    test_dataset = TensorDataset(torch.tensor(test_data, dtype=torch.float32), torch.tensor(test_labels, dtype=torch.long))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

def train_model_RNN(model, train_loader,test_loader, criterion, optimizer, num_epochs, device):
    train_loss = []
    test_loss = []
    
    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = []
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            epoch_train_loss.append(loss.item())
        
        train_loss.append(np.mean(epoch_train_loss))
        
        # Validation phase
        model.eval()
        epoch_val_loss = []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                epoch_val_loss.append(loss.item())
        
        test_loss.append(np.mean(epoch_val_loss))
        
        if (epoch + 1) % 5 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss[-1]}, Val Loss: {test_loss[-1]}')

    return train_loss, test_loss

def evaluate_model_RNN(model, test_loader,test_labels, device):
    model.eval()
    test_predictions = []
    
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(probs, 1)
            test_predictions.extend(predicted.cpu().detach().numpy())
    rnn_accuracy = accuracy_score(test_labels, test_predictions)
    print(f'Accuracy of RNN: {rnn_accuracy}')
    #return test_predictions
    return rnn_accuracy

def plot_loss_RNN(train_loss, test_loss):
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, len(train_loss) + 1), train_loss, label='Train Loss')
    plt.plot(range(1, len(test_loss) + 1), test_loss, label='Test Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

## 5. Classification Sin Redes Neuronales

In [13]:
#código de Domingo Mery
class KDEClassifier(BaseEstimator, ClassifierMixin):
    """Bayesian generative classification based on KDE
    from https://jakevdp.github.io/PythonDataScienceHandbook/05.13-kernel-density-estimation.html

    Parameters
    ----------
    bandwidth : float
        the kernel bandwidth within each class
    kernel : str
        the kernel name, passed to KernelDensity
    """
    def __init__(self, bandwidth=1.0, kernel='gaussian'):
        self.bandwidth = bandwidth
        self.kernel = kernel

    def fit(self, X, y):
        self.classes_ = np.sort(np.unique(y))
        training_sets = [X[y == yi] for yi in self.classes_]
        self.models_ = [KernelDensity(bandwidth=self.bandwidth,
                                      kernel=self.kernel).fit(Xi)
                        for Xi in training_sets]
        self.logpriors_ = [np.log(Xi.shape[0] / X.shape[0])
                           for Xi in training_sets]
        return self

    def predict_proba(self, X):
        logprobs = np.array([model.score_samples(X)
                             for model in self.models_]).T
        result = np.exp(logprobs + self.logpriors_)
        return result / result.sum(1, keepdims=True)

    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), 1)]

In [14]:
def create_classifiers():
    return {
    'knn                               ': KNeighborsClassifier(n_neighbors=1),
    'knn-3                             ': KNeighborsClassifier(n_neighbors=3),
    'knn-5                             ': KNeighborsClassifier(n_neighbors=5),
    'knn-8                             ': KNeighborsClassifier(n_neighbors=8),
    'knn-9                             ': KNeighborsClassifier(n_neighbors=9),
    'knn-10                            ': KNeighborsClassifier(n_neighbors=10),
    'knn-15                            ': KNeighborsClassifier(n_neighbors=15),
    'knn-20                            ': KNeighborsClassifier(n_neighbors=15),
    'mlp                               ': MLPClassifier(alpha=1, max_iter=1000, random_state=42),
    'mlp layers 2                      ': MLPClassifier(hidden_layer_sizes=(100, 50), alpha=1, max_iter=1000, random_state=42),
    'svm lineal 1                      ': SVC(kernel='linear', gamma=0.2, C=0.1),
    'svm lineal 2                      ': SVC(kernel='linear', gamma=0.25, C=0.2),
    'svm polinomial                    ': SVC(kernel='poly', gamma=0.2,degree=3, C=0.1),
    'svm rbf 1                         ': SVC(kernel='rbf', gamma=0.2, C=0.1),
    'svm rbf 2                         ': SVC(kernel='rbf', gamma=0.65, C=0.25),
    'svm rbf 3                         ': SVC(kernel="rbf", random_state=42),
    'svm rbf gamma auto                ': SVC(kernel='rbf', gamma='auto'),
    'svm sigmoidal                     ': SVC(kernel='sigmoid', gamma=0.01, C=1.5),
    'dmin                              ': NearestCentroid(),
    'bayes kde                         ': KDEClassifier(bandwidth=1.0),
    'naive bayes                       ': GaussianNB(),
    'lda                               ': LinearDiscriminantAnalysis(),
    'qda                               ': QuadraticDiscriminantAnalysis(),
    'random forest depth 3             ': RandomForestClassifier(max_depth=3,n_estimators=150),
    'random forest depth 15            ': RandomForestClassifier(max_depth=15,n_estimators=150),
    'random forest depth 30            ': RandomForestClassifier(max_depth=30, n_estimators=200),
    'random forest depth 100           ': RandomForestClassifier(max_depth=100,n_estimators=150),
    'random forest n_estimators 300    ': RandomForestClassifier(n_estimators=300),
    'decision tree                     ': DecisionTreeClassifier(max_depth=3),
    'decision tree depth 12            ': DecisionTreeClassifier(max_depth=12),
    'decision tree depth 100           ': DecisionTreeClassifier(max_depth=100),
    'logistic regression lbfgs         ': LogisticRegression(C=0.1,solver="lbfgs"),
    'logistic regression newton-cg     ': LogisticRegression(C=0.2,solver="newton-cg"),
    'gradient boosting                 ': GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42),
    'adaboost                          ': AdaBoostClassifier(n_estimators=100),
    'extra trees                       ': ExtraTreesClassifier(n_estimators=100),
    'bagging                           ': BaggingClassifier(n_estimators=50),
    'perceptron                        ': Perceptron(max_iter=1000),
    'sgd                               ': SGDClassifier(max_iter=1000, tol=1e-3)
    }

### HO

In [15]:
def clasificacion(model, train_data, train_labels, test_data):
    model.fit(train_data, train_labels) #Se clasifica el Testing
    y_pred = model.predict(test_data)  #Se calcula el Accuracy en el Testing y se almacena en acc[i]
    return y_pred

## Evaluation

### VC

In [16]:
def crossval(clf,X,y,nfolds=10,show=1):
  scores = cross_val_score(clf, X, y, cv=nfolds)
  acc = np.mean(scores)
  if show:
    acc_st = "{:.2f}".format(acc*100)
    print('Accuracy = '+str(acc_st))
  return acc

In [17]:
def evaluate_classifiers(classifiers, features, labels, nfolds=10, use_sfs=False,n_features=20):
    if use_sfs:
        sfs_indices = sfs(features, labels, n_features)
        features = features[:, sfs_indices]
        print(f'Selected Features: {sfs_indices}')

    accuracies = {}
    for name, clf in classifiers.items():
        print(f"\nEvaluando: {name.strip()}")
        accuracies[name] = crossval(clf, features, labels, nfolds=nfolds, show=1)
    
    print("\nPrecisión total por clasificador:")
    for name, acc in accuracies.items():
        print(f'{name.strip()}: {acc:.2f}')
    
    return accuracies

### HO

In [18]:
def print_accuracies(accuracies):
    print("\nAccuracy Total por Clasificador:")
    for name, acc in accuracies.items():
        print(f"{name}: {acc}")

#### Solo clean y normalizar

In [19]:
def evaluate_model_cleannormalize(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = clean_normalized_feature_selection(train_data, test_data)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies

#### sfs

In [20]:
def evaluate_model_sfs5(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = sfs_selection(train_data, test_data, train_labels,5)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    return accuracies

def evaluate_model_sfs10(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = sfs_selection(train_data, test_data, train_labels,10)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies
def evaluate_model_sfs15(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = sfs_selection(train_data, test_data, train_labels,15)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies
def evaluate_model_sfs20(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = sfs_selection(train_data, test_data, train_labels,20)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies
def evaluate_model_sfs25(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = sfs_selection(train_data, test_data, train_labels,25)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies

#### pca

In [21]:
def evaluate_model_pca10(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = pca_tranformer(train_data, test_data,10)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies
def evaluate_model_pca20(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = pca_tranformer(train_data, test_data,20)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies
def evaluate_model_pca25(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = pca_tranformer(train_data, test_data,25)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies

#### ica

In [22]:
def evaluate_model_ica10(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = ica_tranformer(train_data, test_data, train_labels,10)

   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies
def evaluate_model_ica20(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = ica_tranformer(train_data, test_data, train_labels,20)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies
def evaluate_model_ica25(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = ica_tranformer(train_data, test_data, train_labels,25)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies

#### plsr

In [23]:
def evaluate_model_plsr10(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = plsr_transformer(train_data, test_data, train_labels,10)

   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies
def evaluate_model_plsr20(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = plsr_transformer(train_data, test_data, train_labels,20)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies
def evaluate_model_plsr25(train_data, test_data, train_labels, test_labels, classifiers):
    accuracies = {name: [] for name in classifiers.keys()}
    train_data, test_data = plsr_transformer(train_data, test_data, train_labels,25)
   
    for name, clf in classifiers.items():
        ypred = clasificacion(clf, train_data, train_labels, test_data)
        accuracy = accuracy_score(test_labels, ypred)
        accuracies[name].append(accuracy)
    
    return accuracies